In [2]:
import numpy as np
import matplotlib.patches as pat
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
# load the text file
train = pd.read_csv("data/train.csv")
train = np.array(train.values)

In [4]:
X = train[:,0:2] # id & text
Y = train[:,2:] # classes

In [5]:
import string
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer

punctuations = string.punctuation.replace('\'','') + "0123456789"
outtab = "".join([" " for _ in punctuations])
trantab = str.maketrans(punctuations, outtab)
lemmatiser = WordNetLemmatizer()
stemmer = PorterStemmer()

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/hadi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
lim = 0
XC = np.array(X[:,1])
for i in range(X.shape[0]):
    t = X[i,1].lower().translate(trantab).split()
    t = [w for w in t if len(w) < 50]
    l = [stemmer.stem(lemmatiser.lemmatize(w)) for w in t]
    XC[i] = " ".join(l)

In [7]:
# import and instantiate TfidfVectorizer (with the default parameters)
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer()

# use TreeankWordTokenizer
from nltk.tokenize import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()
vect.set_params(tokenizer=tokenizer.tokenize)

# remove English stop words
vect.set_params(stop_words='english')

# include 1-grams and 2-grams
vect.set_params(ngram_range=(1, 2))

# ignore terms that appear in more than 50% of the documents
vect.set_params(max_df=0.5)

# only keep terms that appear in at least 1 documents
vect.set_params(min_df=1)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.5, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<bound method TreebankWordTokenizer.tokenize of <nltk.tokenize.treebank.TreebankWordTokenizer object at 0x7efcf12c7278>>,
        use_idf=True, vocabulary=None)

In [8]:
# vectorization
docs_list = XC

In [9]:
vect_out = vect.fit_transform(docs_list)

In [10]:
import scipy
#scipy.sparse.save_npz('data/train_tfidf.npz', vect_out)
#docs_list = scipy.sparse.load_npz('data/train_tfidf.npz')

In [13]:
# Tokenize and Pad
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential

from keras.layers import CuDNNGRU, Dense, Conv1D, MaxPooling1D
from keras.layers import Dropout, GlobalMaxPooling1D, BatchNormalization
from keras.layers import Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Nadam
# Create tokenizer
tokenizer = Tokenizer(num_words=None,
                      filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                      lower=True,
                      split=" ",
                      char_level=False)
# Fit and run tokenizer
tokenizer.fit_on_texts(list(docs_list))
tokenized_train = tokenizer.texts_to_sequences(docs_list)
#tokenized_test = tokenizer.texts_to_sequences(x_test)
word_index = tokenizer.word_index


embedding_dim = 300
# Get embeddings
embeddings_index = {}
f = open('wiki.en.vec', encoding="utf8")
for line in f:
    values = line.rstrip().rsplit(' ', embedding_dim)
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
# Build embedding matrix
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

InternalError: Could not allocate ndarray

In [17]:
# Initiate model
model = Sequential()
# Add Embedding layer
vocab_size = 157781
max_len = 200
model.add(Embedding(vocab_size + 1, embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=True))
# Add Convolutional layer
model.add(Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'))
model.add(MaxPooling1D(3))
model.add(GlobalMaxPooling1D())
model.add(BatchNormalization())
# Add fully connected layers
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(6, activation='sigmoid'))



# Before
# Tokenize and Pad
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence0 import pad_sequences
# Create tokenizer
tokenizer = Tokenizer(num_words=None,
                      filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                      lower=True,
                      split=" ",
                      char_level=False)
# Fit and run tokenizer
tokenizer.fit_on_texts(list(x_train))
tokenized_train = tokenizer.texts_to_sequences(x_train)
#tokenized_test = tokenizer.texts_to_sequences(x_test)
word_index = tokenizer.word_index

InternalError: Could not allocate ndarray